This notebook authenticates EE, configures a small region/time window, triggers TFRecord exports using data_export.export_ee_data, and shows how to monitor and optionally download results.

In [13]:
import json
from datetime import datetime
from pathlib import Path    

import ee
# Optional for \downloading TFRecords later:
# from google.cloud import storage

from data_export import export_ee_data, ee_utils

In [14]:
try:
    ee.Authenticate()
except Exception as exc:
    print("Auth skipped or already configured:", exc)

In [15]:
ee.Initialize()
print("Earth Engine initialized", datetime.utcnow())

Earth Engine initialized 2025-11-20 23:55:16.010253


In [16]:
params = {
    "bucket": "lmudl-wildfire-compilation-bucket",     # e.g., "my-bucket"
    "folder": "eaton",                 # subfolder inside bucket
    "prefix": "eaton_sample",               # file prefix
    "start_date": "2025-1-05",
    "end_date": "2025-01-13",
    "kernel_size": 64,
    "sampling_scale": 1000,
    "eval_split_ratio": 0.05,
    "num_samples_per_file": 100,
    "region_bbox": [-118.19466979980469, 34.147511224015496, -118.00034912109375, 34.24916929738899]
}
print(json.dumps(params, indent=2))

{
  "bucket": "lmudl-wildfire-compilation-bucket",
  "folder": "eaton",
  "prefix": "eaton_sample",
  "start_date": "2025-1-05",
  "end_date": "2025-01-13",
  "kernel_size": 64,
  "sampling_scale": 1000,
  "eval_split_ratio": 0.05,
  "num_samples_per_file": 100,
  "region_bbox": [
    -118.19466979980469,
    34.147511224015496,
    -118.00034912109375,
    34.24916929738899
  ]
}


In [17]:
def export_slice(config: dict):
    required = ["bucket", "folder", "prefix", "start_date", "end_date"]
    for key in required:
        if not config.get(key):
            raise ValueError(f"Missing required parameter: {key}")

    bbox = config.get("region_bbox") or ee_utils.COORDINATES["US"]
    # Override the default region used by export_ee_data
    ee_utils.COORDINATES["US"] = bbox

    start_date = ee.Date(config["start_date"])
    end_date = ee.Date(config["end_date"])
    
    # Convert bbox to ee.Geometry.Rectangle
    geometry = ee.Geometry.Rectangle(bbox)

    export_ee_data.export_single_fire_dataset(
        bucket=config["bucket"],
        folder=config["folder"],
        start_date=start_date,
        end_date=end_date,
        geometry=geometry,
        prefix=config.get("prefix", ""),
        kernel_size=config.get("kernel_size", ee_utils.DEFAULT_KERNEL_SIZE),
        sampling_scale=config.get("sampling_scale", ee_utils.DEFAULT_SAMPLING_RESOLUTION),
        num_samples_per_file=config.get("num_samples_per_file", ee_utils.DEFAULT_LIMIT_PER_EE_CALL),
    )
    print("Export triggered. Check https://code.earthengine.google.com/tasks for progress.")

In [18]:
export_slice(params)

Export triggered. Check https://code.earthengine.google.com/tasks for progress.


In [19]:
def list_tasks(limit: int = 10):
    tasks = ee.batch.Task.list()
    for task in tasks[:limit]:
        status = task.status()
        print(f"{status['id']} | {status.get('state')} | {status.get('description')}")

list_tasks()

VXWRIXZ56WXZ3AVMWPFUNCAD | READY | eaton_sample_000
LJYAJVGK4Q2MD5KMQYCAHYJ2 | COMPLETED | eaton_sample_000
MKWLPT3U5CLXWJ7ZQQJDLFMO | COMPLETED | eaton_sample_000
OAC5GVNWEL55HM7QL2SX6UDU | COMPLETED | eaton_sample_000
Z5GP6Q6XR44PZ2GJKJOBIL3M | COMPLETED | eaton_sample_000
BUFZVNCYGKUYIWBLN64I3LT4 | COMPLETED | eaton_sample_000
WN4ESNCY24DRGMN5XOGM33CI | COMPLETED | eaton_sample_000
DCJE22GGK7MIGBOHSW2PVIIZ | COMPLETED | palisades_sample_000
NASXEXBBLLOD5AGY5C5XQD5K | COMPLETED | palisades_sample_000
NH523CSAUBOIGD6PA6OF25NY | COMPLETED | eaton_sample_000


In [20]:
from google.cloud import storage

def download_exports(config: dict, destination: Path):
    destination.mkdir(parents=True, exist_ok=True)
    client = storage.Client()
    bucket = client.bucket(config["bucket"])

    folder = str(config.get("folder", "")).strip("/")
    gcs_prefix = (folder + "/") if folder else ""

    blobs = list(bucket.list_blobs(prefix=gcs_prefix))

    print(f"Found {len(blobs)} files under gs://{config['bucket']}/{gcs_prefix}")
    for blob in blobs:
        target = destination / Path(blob.name).name
        print("Downloading", blob.name, "->", target)
        blob.download_to_filename(target)


In [21]:
%%!
gcloud auth application-default login

^C


[]

In [23]:
download_exports(params, Path("exports"))

Found 1 files under gs://lmudl-wildfire-compilation-bucket/eaton/
